In [7]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [19]:
missing_list = ['Tom Brady', 'Drew Brees', 'Peyton Manning', 'Brett Favre', 'Ben Roethlisberger', 'Philip Rivers', 
                'Matt Ryan', 'Dan Marino', 'Aaron Rodgers', 'Matthew Stafford', 'Eli Manning', 'John Elway', 
                'Warren Moon', 'Fran Tarkenton', 'Carson Palmer', 'Vinny Testaverde', 'Drew Bledsoe', 'Joe Flacco', 
                'Russell Wilson', 'Dan Fouts', 'Kerry Collins', 'Kirk Cousins', 'Joe Montana', 'Johnny Unitas', 
                'Derek Carr', 'Andy Dalton', 'Dave Krieg', 'Boomer Esiason', 'Donovan McNabb', 'Matt Hasselbeck', 
                'Alex Smith', 'Jim Kelly', 'Jay Cutler', 'Ryan Fitzpatrick', 'Ryan Tannehill', 'Jim Everett', 
                'Jim Hart', 'Steve Deberg', 'Tony Romo', 'John Hadl', 'Phil Simms', 'Steve Young', 'Y.A. Tittle', 
                'Troy Aikman', 'Ken Anderson', 'Cam Newton', 'Kurt Warner', 'Sonny Jurgensen', 'Mark Brunell', 
                'John Brodie', 'Jared Goff', 'Steve McNair', 'Dak Prescott', 'Norm Snead', 'Randall Cunningham', 
                'Joe Ferguson', 'Jon Kitna', 'Patrick Mahomes', 'Roman Gabriel', 'Jake Plummer', 'Brad Johnson', 
                'Rich Gannon', 'Len Dawson', 'Chris Chandler', 'Trent Green', 'Ron Jaworski', 'Terry Bradshaw', 
                'Ken Stabler', 'Craig Morton', 'Joe Namath', 'Jeff George', 'George Blanda', 'Steve Grogan', 
                'Bobby Layne', 'Jim Harbaugh', 'Jim Plunkett', 'Jeff Garcia', 'Matt Schaub', 'Joe Theismann', 
                "Ken O'Brien", 'Bob Griese', 'Tommy Kramer', 'Bart Starr', 'Charley Johnson', 'Daunte Culpepper', 
                'Billy Joe Tolliver', 'Arnie Herber', 'Paul Christman', 'Benny Friedman', 'Cecil Isbell', 
                'Frank Filchock', 'Jacky Lee', 'Glenn Dobbs', 'Roy Zimmerman', 'Parker Hall', 'Fred Enke', 
                'Irv Comp', "Charlie O'Rourke", 'Ace Parker', 'Bud Schwenk', 'Bob Lee', 'George Izo', 'C.J. Stroud', 
                'J.J. McCarthy', 'Bo Nix', 'E.J. Manuel', 'Matt McGloin', 'Tanner Lee', 'Jameill Showers', 
                'Joel Stave', 'Mike Bercovici', 'Bryn Renner', 'John Rhys Plumlee', "J'Mar Smith", 'Jake Heaps', 
                'Wilton Speight', 'Chevan Cordeiro', 'Brock Jensen', 'Luis Perez', 'Peewee Jarrett', 'Nic Shimonek', 
                'James Blackman', 'Casey Bauman', 'Rocky Lombardi', 'Riley Neal', 'Nolan Henderson', 'John Paddock', 
                'Brendon Kay', 'Logan Kilgore', 'Jerry Lovelocke', 'Peter Pujals', 'Max Wittek', 'Nick Schuessler', 
                'Zack Annexstad', 'Kory Faulkner', 'Tim Demorat', 'Travis Partridge', 'Dalton Sturm', 'Tommy Rees', 
                'Broc Rutter']



In [20]:
# Import CSV   pro_ref_individ_raw.csv and get the names we captured from the URL scrape.
import pandas as pd
rel_path = "../Data_Artifacts/pro_ref_individ_raw.csv"
individ_df = pd.read_csv(rel_path)
individ_names = individ_df['name'].tolist()

# Get a fuller list of Quarterbacks to get the ones we need from this site:

display(f'individ: {len(individ_names)}')
display(f'missing {len(missing_list)}')
# Find the intersection
intersection = list(set(individ_names) & set(missing_list))
print("Intersection of individ_names and missing_list:", intersection)

'individ: 497'

'missing 140'

Intersection of individ_names and missing_list: []


In [21]:
def get_recognition(soup):
    recognition_data = {}
    try:
        bling = soup.find('ul', id='bling')
        recognition_data['all_star'] = [li.get_text(strip=True) for li in bling.find_all('li', class_='all_star')]
        recognition_data['uniforms'] = [a['data-tip'] for a in soup.find('div', class_='uni_holder').find_all('a')]
    except AttributeError:
        recognition_data = {key: "N/A" for key in ['all_star', 'uniforms']}
    return recognition_data

def get_summary_stats(soup):
    summary_stats = {}
    try:
        summary = soup.find('div', {'class': 'stats_pullout'})
        summary_stats['games_played'] = summary.find('span', string='G').find_next('p').text.strip()
        summary_stats['approximate_value'] = summary.find('span', string='AV').find_next('p').text.strip()
        summary_stats['qb_record'] = summary.find('span', string='QBrec').find_next('p').text.strip()
        summary_stats['completion_percentage'] = summary.find('span', string='Cmp%').find_next('p').text.strip()
        summary_stats['passing_yards'] = summary.find('span', string='Yds').find_next('p').text.strip()
        summary_stats['yards_per_attempt'] = summary.find('span', string='Y/A').find_next('p').text.strip()
        summary_stats['touchdowns'] = summary.find('span', string='TD').find_next('p').text.strip()
        summary_stats['interceptions'] = summary.find('span', string='Int').find_next('p').text.strip()
        summary_stats['fantasy_points'] = summary.find('span', string='FantPt').find_next('p').text.strip()
    except AttributeError:
        summary_stats = {key: "N/A" for key in ['games_played', 'approximate_value', 'qb_record', 'completion_percentage', 'passing_yards', 'yards_per_attempt', 'touchdowns', 'interceptions', 'fantasy_points']}
    return summary_stats

In [22]:
import re

def get_faq(soup):
    faq_data = {}
    keyword_patterns = {
        'when_born': re.compile(r'when.*born', re.IGNORECASE),
        'where_born': re.compile(r'where.*born', re.IGNORECASE),
        'height': re.compile(r'tall', re.IGNORECASE),
        'weight': re.compile(r'weigh', re.IGNORECASE),
        'games_played': re.compile(r'games.*play', re.IGNORECASE),
        'pass_yards': re.compile(r'passing.*yards', re.IGNORECASE),
        'TD': re.compile(r'touchdowns', re.IGNORECASE),
        'superbowls': re.compile(r'Super Bowls', re.IGNORECASE),
        'retire_year': re.compile(r'retire', re.IGNORECASE),
        'nickname': re.compile(r'nicknames', re.IGNORECASE)
    }

    try:
        faq = soup.find('div', id='div_faq')
        for key, pattern in keyword_patterns.items():
            question = faq.find('h3', string=pattern)
            if question:
                answer = question.find_next('p').text
                if key in ['when_born', 'where_born']:
                    faq_data[key] = re.split(r'on | in ', answer)[-1].strip('.')
                elif key == 'height':
                    faq_data[key] = re.search(r'\d+-\d+', answer).group()
                elif key == 'weight':
                    faq_data[key] = re.search(r'\d+ lbs', answer).group().split()[0]
                elif key == 'superbowls':
                    faq_data[key] = re.search(r'\d+', answer) and re.search(r'\d+', answer).group() or '0'
                elif key in ['games_played', 'pass_yards', 'TD']:
                    faq_data[key] = re.search(r'\d+', answer.replace(',', '')).group()
                elif key == 'retire_year':
                    faq_data[key] = re.search(r'\d+', answer).group()
                elif key == 'nickname':
                    faq_data[key] = re.split(r' is a nickname| are nicknames', answer)[0]
                else:
                    faq_data[key] = answer
    except AttributeError as e:
        print(f"An error occurred: {e}")
    return faq_data

In [23]:
import re

def get_qb_header_data(soup):
    qb_header_data = {
        'throws': '', 'hall_of_fame': '', 'college': '', 'high_school': '',
        'weighted_career_av': '', 'college_stats': '', 'draft_team': '',
        'draft_round': '', 'draft_pick': '', 'draft_year': ''    }

    try:
        # Extract Throws
        throws_tag = soup.find('strong', string='Throws:')
        if throws_tag and throws_tag.next_sibling:
            qb_header_data['throws'] = throws_tag.next_sibling.strip()

        # Extract College
        college_tag = soup.find('strong', text='College')
        if college_tag:
            college = college_tag.find_next('a')
            if college:
                qb_header_data['college'] = college.text.strip()

        # Extract College Stats URL
        college_stats_tag = soup.find('a', href=re.compile(r'https://www.sports-reference.com/cfb/players/'))
        if college_stats_tag:
            qb_header_data['college_stats'] = college_stats_tag['href']

        # Extract Weighted Career AV (wAV)
        wav_tag = re.search(r'Weighted Career AV.*?(\d+)', str(soup))
        if wav_tag:
            qb_header_data['weighted_career_av'] = wav_tag.group(1)

        # Extract High School
        high_school_tag = soup.find('strong', text='High School')
        if high_school_tag:
            high_school = high_school_tag.find_next('a')
            if high_school:
                qb_header_data['high_school'] = high_school.text.strip()

        # Extract Hall of Fame year (hof)
        hof_tag = re.search(r'Hall of Fame.*?(\d{4})', str(soup))
        if hof_tag:
            qb_header_data['hall_of_fame'] = hof_tag.group(1)

        # Extract Draft Team, Round, Pick, and Year
        draft_team_tag = re.search(r'Draft.*?<a.*?>(.*?)</a>', str(soup))
        if draft_team_tag:
            qb_header_data['draft_team'] = draft_team_tag.group(1)

        draft_round_tag = re.search(r'in the (\d+)(?:st|nd|rd|th) round', str(soup))
        if draft_round_tag:
            qb_header_data['draft_round'] = draft_round_tag.group(1)

        draft_pick_tag = re.search(r'\((\d+)(?:st|nd|rd|th) overall\)', str(soup))
        if draft_pick_tag:
            qb_header_data['draft_pick'] = draft_pick_tag.group(1)

        draft_year_tag = re.search(r'(\d{4}) NFL Draft', str(soup))
        if draft_year_tag:
            qb_header_data['draft_year'] = draft_year_tag.group(1)

    except AttributeError as e:
        print(f"An error occurred: {e}")

    return qb_header_data

In [24]:
import requests
from bs4 import BeautifulSoup

def get_soup(url):
    response = requests.get(url)
    if response.status_code == 200:
        return BeautifulSoup(response.content, 'html.parser')
    else:
        print(f"Failed to retrieve webpage. Status code: {response.status_code}")
        return None

url = "https://www.pro-football-reference.com/players/M/MannPe00.htm"

soup = get_soup(url)
if soup:
    qb_header_data = get_qb_header_data(soup)
    summary_stats = get_summary_stats(soup)
    recognition_data = get_recognition(soup)
    faq_data = get_faq(soup)
    print("")
    print("qb_header_data:", qb_header_data)
    print("Summary Stats:", summary_stats)
    print("Recognition:", recognition_data)
    print("FAQ Data:", faq_data)
    print("")
else: 
    print('no soup for you.')

import pprint
# Create a PrettyPrinter instance
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(qb_header_data)

C:\Users\black\AppData\Local\Temp\ipykernel_10512\4017800806.py:16: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  college_tag = soup.find('strong', text='College')
C:\Users\black\AppData\Local\Temp\ipykernel_10512\4017800806.py:33: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  high_school_tag = soup.find('strong', text='High School')



qb_header_data: {'throws': 'Right', 'hall_of_fame': '2000', 'college': 'Tennessee', 'high_school': 'Isidore Newman', 'weighted_career_av': '100', 'college_stats': 'https://www.sports-reference.com/cfb/players/peyton-manning-1.html', 'draft_team': 'Indianapolis Colts', 'draft_round': '1', 'draft_pick': '1', 'draft_year': '1998'}
Summary Stats: {'games_played': '266', 'approximate_value': '271', 'qb_record': '186-79-0', 'completion_percentage': '65.3', 'passing_yards': '71940', 'yards_per_attempt': '7.7', 'touchdowns': '539', 'interceptions': '251', 'fantasy_points': '4686.3'}
Recognition: {'all_star': ['14x Pro Bowl', '7x All-Pro'], 'uniforms': ['Indianapolis Colts 1998-2010', 'Denver Broncos 2012-2015']}
FAQ Data: {'when_born': 'March 24, 1976', 'where_born': 'New Orleans, LA', 'height': '6-5', 'weight': '230', 'games_played': '266', 'pass_yards': '71940', 'TD': '539', 'superbowls': '2', 'retire_year': '2015', 'nickname': 'The Sheriff, The Caveman, Peydirt'}

{   'college': 'Tennessee

In [25]:
base_url = "https://www.pro-football-reference.com/players/"
suffix_url = "00.htm"

def create_url(name):
    '''function to build the following url from the name passed in:
    name_link=  <first letter of last name> / <first four letters of last name><first two letters of first name>
    eg.  Kurt Warner --> W/WarnKu
    Kurt Warner, https://www.pro-football-reference.com/players/W/WarnKu00.htm'''

    try:
        # Remove apostrophes from the name
        name = name.replace("'", "")
        name_parts = name.split()

        # Ensure there are at least two parts (first name and last name)
        if len(name_parts) < 2:
            raise ValueError("Name must contain at least a first and last name")

        # The first part is the first name, and the last part is the last name
        first_name = name_parts[0]  
        # Combine all parts of the last name (if there are multiple parts)
        last_name = ''.join(name_parts[1:])

        # The directory is based on the first letter of the last name
        directory_letter = last_name[0].upper()

        # Ensure the last name has at least 4 characters to avoid index errors
        if len(last_name) < 4:
            raise ValueError("Last name must have at least 4 characters for URL creation")

        # Create the name link using the first four letters of the last name and the first two letters of the first name
        name_link = f"{last_name[:4]}{first_name[:2]}"

        # Build the final URL
        url = f"https://www.pro-football-reference.com/players/{directory_letter}/{name_link}00.htm"

        return url

    except (IndexError, ValueError) as e:
        # In case of any error, return a placeholder or error message
        print(f"Error generating URL for '{name}': {e}")
        return "https://www.pro-football-reference.com/players/Unknown/Unknown00.htm"

# name_list = ["Steve Deberg",      # Expected: players/D/DebeSt00.htm
#                 "Norman Van Broklin",  # Expected: VanBroNo
#                 "Kurt Warner",         # Expected: WarnKu
#                 "Ken O'Brien",     # Expected: players/O/OBriKe00.htm
#                 "John Smith"]           # Expected: SmitJo 

# for name in name_list: 
#     print(create_url(name))

In [26]:
import time

quarterback_dict = {}
quarterback_fails = []
qb_list = missing_list #['Steve Deberg']
print(len(qb_list))

# Combine data from each dictionary into a single player_data dictionary
def merge_player_data_dicts(name):
    player_data = {}

    player_data.update(qb_header_data)       # Add QB header data
    player_data.update(summary_stats)         # Add summary stats
    player_data.update(recognition_data)      # Add recognition data
    player_data.update(faq_data)              # Add FAQ data

    # Store the player_data dictionary in the quarterback_dict, with player's name as key
    quarterback_dict[name] = player_data

for idx, name in enumerate(qb_list):
    print(f'player ({idx}) :  {name}')
    time.sleep(8)
    soup = get_soup(create_url(name))
    if soup:
        qb_header_data = get_qb_header_data(soup)
        summary_stats = get_summary_stats(soup)
        recognition_data = get_recognition(soup)
        faq_data = get_faq(soup)
        print("qb_header_data:", qb_header_data)
        print("Summary Stats:", summary_stats)
        print("Recognition:", recognition_data)
        print("FAQ Data:", faq_data)
        print("")
        merge_player_data_dicts(name)
    else: 
        quarterback_fails.append(name)     
        print(f'no soup for you, {name}')  
        print("")


# output is the quarterback_dict, where a ton of variables for each Quarterback
#   are accessed with the Quarterback name as the key.

140
player (0) :  Tom Brady


C:\Users\black\AppData\Local\Temp\ipykernel_10512\4017800806.py:16: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  college_tag = soup.find('strong', text='College')
C:\Users\black\AppData\Local\Temp\ipykernel_10512\4017800806.py:33: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  high_school_tag = soup.find('strong', text='High School')


qb_header_data: {'throws': 'Right', 'hall_of_fame': '2000', 'college': 'Michigan', 'high_school': 'Junipero Serra', 'weighted_career_av': '100', 'college_stats': 'https://www.sports-reference.com/cfb/players/tom-brady-1.html', 'draft_team': 'New England Patriots', 'draft_round': '6', 'draft_pick': '199', 'draft_year': '2000'}
Summary Stats: {'games_played': '335', 'approximate_value': '326', 'qb_record': '251-82-0', 'completion_percentage': '64.3', 'passing_yards': '89214', 'yards_per_attempt': '7.4', 'touchdowns': '649', 'interceptions': '212', 'fantasy_points': '5943.4'}
Recognition: {'all_star': ['15x Pro Bowl', '3x All-Pro'], 'uniforms': ['New England Patriots 2000-2019', 'Tampa Bay Buccaneers 2020-2022']}
FAQ Data: {'when_born': 'August 3, 1977', 'where_born': 'San Mateo, CA', 'height': '6-4', 'weight': '225', 'games_played': '335', 'pass_yards': '89214', 'TD': '649', 'superbowls': '7', 'retire_year': '2022', 'nickname': 'TB12, Tom Terrific, Touchdown Tom, GOAT, The Pharaoh, Comeb

In [27]:
display(quarterback_dict)
display(quarterback_fails)

{'Tom Brady': {'throws': 'Right',
  'hall_of_fame': '2000',
  'college': 'Michigan',
  'high_school': 'Junipero Serra',
  'weighted_career_av': '100',
  'college_stats': 'https://www.sports-reference.com/cfb/players/tom-brady-1.html',
  'draft_team': 'New England Patriots',
  'draft_round': '6',
  'draft_pick': '199',
  'draft_year': '2000',
  'games_played': '335',
  'approximate_value': '326',
  'qb_record': '251-82-0',
  'completion_percentage': '64.3',
  'passing_yards': '89214',
  'yards_per_attempt': '7.4',
  'touchdowns': '649',
  'interceptions': '212',
  'fantasy_points': '5943.4',
  'all_star': ['15x Pro Bowl', '3x All-Pro'],
  'uniforms': ['New England Patriots 2000-2019',
   'Tampa Bay Buccaneers 2020-2022'],
  'when_born': 'August 3, 1977',
  'where_born': 'San Mateo, CA',
  'height': '6-4',
  'weight': '225',
  'pass_yards': '89214',
  'TD': '649',
  'superbowls': '7',
  'retire_year': '2022',
  'nickname': 'TB12, Tom Terrific, Touchdown Tom, GOAT, The Pharaoh, Comeback K

['Billy Joe Tolliver',
 'Arnie Herber',
 'Paul Christman',
 'Benny Friedman',
 'Cecil Isbell',
 'Frank Filchock',
 'Jacky Lee',
 'Glenn Dobbs',
 'Roy Zimmerman',
 'Parker Hall',
 'Fred Enke',
 'Irv Comp',
 "Charlie O'Rourke",
 'Ace Parker',
 'Bud Schwenk',
 'Bob Lee',
 'George Izo',
 'C.J. Stroud',
 'J.J. McCarthy',
 'Bo Nix',
 'E.J. Manuel',
 'Matt McGloin',
 'Tanner Lee',
 'Jameill Showers',
 'Joel Stave',
 'Mike Bercovici',
 'Bryn Renner',
 'John Rhys Plumlee',
 "J'Mar Smith",
 'Jake Heaps',
 'Wilton Speight',
 'Chevan Cordeiro',
 'Brock Jensen',
 'Luis Perez',
 'Peewee Jarrett',
 'Nic Shimonek',
 'James Blackman',
 'Casey Bauman',
 'Rocky Lombardi',
 'Riley Neal',
 'Nolan Henderson',
 'John Paddock',
 'Brendon Kay',
 'Logan Kilgore',
 'Jerry Lovelocke',
 'Peter Pujals',
 'Max Wittek',
 'Nick Schuessler',
 'Zack Annexstad',
 'Kory Faulkner',
 'Tim Demorat',
 'Travis Partridge',
 'Dalton Sturm',
 'Tommy Rees',
 'Broc Rutter']

In [40]:
# Put the nested Dictionary quaterback_dict into a DataFrame
# 1. Convert quarterback_dict to a list of dictionaries, where each dictionary is a row
# 2. For each name, put their row of data in.
# 3. Create a Dataframe

data_list = []     # holds the dictionaries where each dictionary represents a player's data, with 'name' as the first key

for name, stats in quarterback_dict.items():
    # print(f'{name} \n {stats}')
    player_row = {'name': name}
    player_row.update(stats)  # Add the stats to the player_row dictionary
    # print(player_row)
    data_list.append(player_row)
individ_df = pd.DataFrame(data_list)   # converts the list of dictionaries into a Pandas DataFrame
display(individ_df.head())

fails_df = pd.DataFrame(quarterback_fails)   # list of names
display(fails_df.head())

,name,throws,hall_of_fame,college,high_school,weighted_career_av,college_stats,draft_team,draft_round,draft_pick,...,uniforms,when_born,where_born,height,weight,pass_yards,TD,superbowls,retire_year,nickname
0,Tom Brady,Right,2000,Michigan,Junipero Serra,100,https://www.sports-reference.com/cfb/players/t...,New England Patriots,6,199,...,"[New England Patriots 2000-2019, Tampa Bay Buc...","August 3, 1977","San Mateo, CA",6-4,225,89214,649,7,2022,"TB12, Tom Terrific, Touchdown Tom, GOAT, The P..."
1,Drew Brees,Right,8842,Purdue,Westlake,100,https://www.sports-reference.com/cfb/players/d...,San Diego Chargers,2,32,...,"[San Diego Chargers 2001-2005, New Orleans Sai...","January 15, 1979","Austin, TX",6-0,209,80358,571,1,2020,NaN
2,Peyton Manning,Right,2000,Tennessee,Isidore Newman,100,https://www.sports-reference.com/cfb/players/p...,Indianapolis Colts,1,1,...,"[Indianapolis Colts 1998-2010, Denver Broncos ...","March 24, 1976","New Orleans, LA",6-5,230,71940,539,2,2015,"The Sheriff, The Caveman, Peydirt"
3,Brett Favre,Right,1990,Southern Miss,Hancock North Central,100,https://www.sports-reference.com/cfb/players/b...,Atlanta Falcons,2,33,...,"[Atlanta Falcons 1991, Green Bay Packers 1992-...","October 10, 1969","Gulfport, MS",6-2,222,71838,508,1,2010,"Country, Gunslinger, Buckhead Brett"
4,Ben Roethlisberger,Right,8842,Miami (OH),Findlay,100,https://www.sports-reference.com/cfb/players/b...,Pittsburgh Steelers,1,11,...,[Pittsburgh Steelers 2004-2021],"March 2, 1982","Lima, OH",6-5,240,64088,418,2,2021,Big Ben


,0
0,Billy Joe Tolliver
1,Arnie Herber
2,Paul Christman
3,Benny Friedman
4,Cecil Isbell


# # PAUSE POINT: save dataframes dictionary to pickle and / or CSV

In [42]:
#Export csv
rel_path = "../Data_Artifacts/pro_ref_individ_missing_raw.csv"
individ_df.to_csv(rel_path, index=False)

rel_path = "../Data_Artifacts/pro_ref_individ_missing_fails_raw.csv"
fails_df.to_csv(rel_path, index=False)

In [30]:
#  NOT USED
# # Save as Pickle; pickle.dump to store data

# path = '../Data_Artifacts/'
# with open(f'{path}pro_ref_individ_missing.pkl', 'wb') as file:
#     pickle.dump(quarterback_dict, file)

# path = '../Data_Artifacts/'
# with open(f'{path}pro_ref_individ_missing_fails.pkl', 'wb') as file2:
#     pickle.dump(quarterback_fails, file2)

#     import pickle
# path = '../Data_Artifacts/'

# with open(f'{path}pro_ref_individ_missing.pkl', 'rb') as f:
#     quarterback_dict_load = pickle.load(f)

# with open(f'{path}pro_ref_individ_missing_fails.pkl', 'rb') as f:
#     quarterback_missing_load = pickle.load(f)
    

### RESUME POINT: IMPORT CSV for Missing data and combine with the original individ data:

In [44]:
# Import CSV   pro_ref_individ_raw.csv
import pandas as pd

rel_path_1 = "../Data_Artifacts/pro_ref_individ_missing_raw.csv"
rel_path_2 = "../Data_Artifacts/pro_ref_individ_missing_fails_raw.csv"
individ_df = pd.read_csv(rel_path_1)
individ_missing_df = pd.read_csv(rel_path_2)

display(individ_df.iloc[:,:10].head())
display(individ_df.iloc[:,10:20].head())
display(individ_df.iloc[:,20:].head())

display(individ_missing_df.head())

,name,throws,hall_of_fame,college,high_school,weighted_career_av,college_stats,draft_team,draft_round,draft_pick
0,Tom Brady,Right,2000.0,Michigan,Junipero Serra,100,https://www.sports-reference.com/cfb/players/t...,New England Patriots,6.0,199.0
1,Drew Brees,Right,8842.0,Purdue,Westlake,100,https://www.sports-reference.com/cfb/players/d...,San Diego Chargers,2.0,32.0
2,Peyton Manning,Right,2000.0,Tennessee,Isidore Newman,100,https://www.sports-reference.com/cfb/players/p...,Indianapolis Colts,1.0,1.0
3,Brett Favre,Right,1990.0,Southern Miss,Hancock North Central,100,https://www.sports-reference.com/cfb/players/b...,Atlanta Falcons,2.0,33.0
4,Ben Roethlisberger,Right,8842.0,Miami (OH),Findlay,100,https://www.sports-reference.com/cfb/players/b...,Pittsburgh Steelers,1.0,11.0


,draft_year,games_played,approximate_value,qb_record,completion_percentage,passing_yards,yards_per_attempt,touchdowns,interceptions,fantasy_points
0,2000.0,335,326,251-82-0,64.3,89214.0,7.4,649.0,212.0,5943.4
1,2001.0,287,277,172-114-0,67.7,80358.0,7.6,571.0,243.0,5185.0
2,1998.0,266,271,186-79-0,65.3,71940.0,7.7,539.0,251.0,4686.3
3,1991.0,302,259,186-112-0,62.0,71838.0,7.1,508.0,336.0,4371.2
4,2004.0,249,208,165-81-1,64.4,64088.0,7.6,418.0,211.0,4019.5


,all_star,uniforms,when_born,where_born,height,weight,pass_yards,TD,superbowls,retire_year,nickname
0,"['15x Pro Bowl', '3x All-Pro']","['New England Patriots 2000-2019', 'Tampa Bay ...","August 3, 1977","San Mateo, CA",6-4,225,89214.0,649.0,7.0,2022.0,"TB12, Tom Terrific, Touchdown Tom, GOAT, The P..."
1,"['13x Pro Bowl', '1x All-Pro']","['San Diego Chargers 2001-2005', 'New Orleans ...","January 15, 1979","Austin, TX",6-0,209,80358.0,571.0,1.0,2020.0,NaN
2,"['14x Pro Bowl', '7x All-Pro']","['Indianapolis Colts 1998-2010', 'Denver Bronc...","March 24, 1976","New Orleans, LA",6-5,230,71940.0,539.0,2.0,2015.0,"The Sheriff, The Caveman, Peydirt"
3,"['11x Pro Bowl', '3x All-Pro']","['Atlanta Falcons 1991', 'Green Bay Packers 19...","October 10, 1969","Gulfport, MS",6-2,222,71838.0,508.0,1.0,2010.0,"Country, Gunslinger, Buckhead Brett"
4,['6x Pro Bowl'],['Pittsburgh Steelers 2004-2021'],"March 2, 1982","Lima, OH",6-5,240,64088.0,418.0,2.0,2021.0,Big Ben


,0
0,Billy Joe Tolliver
1,Arnie Herber
2,Paul Christman
3,Benny Friedman
4,Cecil Isbell


### NEW DATAFRAME

In [45]:
# Assuming individ_df is your DataFrame
cleaning_df = individ_df.copy()
cleaning_df = cleaning_df[[
    'name', 'hall_of_fame', 'all_star', 'superbowls',     
    'weighted_career_av', 'approximate_value', 'qb_record', 'games_played', 
    'pass_yards', 'TD', 'completion_percentage', 
    'yards_per_attempt', 'interceptions', 
    'draft_year', 'draft_team', 'draft_round', 'draft_pick', 
    'height', 'weight', 'throws'
]]

cleaning_df.columns

Index(['name', 'hall_of_fame', 'all_star', 'superbowls', 'weighted_career_av',
       'approximate_value', 'qb_record', 'games_played', 'pass_yards', 'TD',
       'completion_percentage', 'yards_per_attempt', 'interceptions',
       'draft_year', 'draft_team', 'draft_round', 'draft_pick', 'height',
       'weight', 'throws'],
      dtype='object')

In [46]:
cleaning_df.rename(columns={
                    'name': 'player',
                    'hall_of_fame': 'hof',
                    'all_star': 'all_star',
                    'superbowls': 'SBs',
                    'weighted_career_av': 'wAV',
                    'approximate_value': 'AV',
                    'qb_record': 'record',
                    'games_played': 'games',
                    'pass_yards': 'pass_yards',
                    'TD': 'TD',
                    'completion_percentage': 'comp_%',
                    'yards_per_attempt': 'YPA',
                    'interceptions': 'int',
                    'draft_year': 'drafted',
                    'draft_team': 'team',
                    'draft_round': 'rd',
                    'draft_pick': 'pick',
                    'height': 'ht',
                    'weight': 'wt',
                    'throws': 'hand'
                }, inplace=True)

display(cleaning_df.columns)
display(cleaning_df.iloc[:,:10].head())
display(cleaning_df.iloc[:,10:].head())

Index(['player', 'hof', 'all_star', 'SBs', 'wAV', 'AV', 'record', 'games',
       'pass_yards', 'TD', 'comp_%', 'YPA', 'int', 'drafted', 'team', 'rd',
       'pick', 'ht', 'wt', 'hand'],
      dtype='object')

,player,hof,all_star,SBs,wAV,AV,record,games,pass_yards,TD
0,Tom Brady,2000.0,"['15x Pro Bowl', '3x All-Pro']",7.0,100,326,251-82-0,335,89214.0,649.0
1,Drew Brees,8842.0,"['13x Pro Bowl', '1x All-Pro']",1.0,100,277,172-114-0,287,80358.0,571.0
2,Peyton Manning,2000.0,"['14x Pro Bowl', '7x All-Pro']",2.0,100,271,186-79-0,266,71940.0,539.0
3,Brett Favre,1990.0,"['11x Pro Bowl', '3x All-Pro']",1.0,100,259,186-112-0,302,71838.0,508.0
4,Ben Roethlisberger,8842.0,['6x Pro Bowl'],2.0,100,208,165-81-1,249,64088.0,418.0


,comp_%,YPA,int,drafted,team,rd,pick,ht,wt,hand
0,64.3,7.4,212.0,2000.0,New England Patriots,6.0,199.0,6-4,225,Right
1,67.7,7.6,243.0,2001.0,San Diego Chargers,2.0,32.0,6-0,209,Right
2,65.3,7.7,251.0,1998.0,Indianapolis Colts,1.0,1.0,6-5,230,Right
3,62.0,7.1,336.0,1991.0,Atlanta Falcons,2.0,33.0,6-2,222,Right
4,64.4,7.6,211.0,2004.0,Pittsburgh Steelers,1.0,11.0,6-5,240,Right


In [47]:
# split out the record column into two new columns: wins/losses:
import numpy as np

def split_record(record):
    if pd.isna(record):
        return pd.Series([np.nan, np.nan])
    else:
        parts = record.split('-')
        if len(parts) >= 2 and parts[0].isdigit() and parts[1].isdigit():
            return pd.Series([int(parts[0]), int(parts[1])])
        else:
            return pd.Series([np.nan, np.nan])

# Apply the function to the record column
cleaning_df[['wins', 'loss']] = cleaning_df['record'].apply(split_record)
cleaning_df.head()


,player,hof,all_star,SBs,wAV,AV,record,games,pass_yards,TD,...,int,drafted,team,rd,pick,ht,wt,hand,wins,loss
0,Tom Brady,2000.0,"['15x Pro Bowl', '3x All-Pro']",7.0,100,326,251-82-0,335,89214.0,649.0,...,212.0,2000.0,New England Patriots,6.0,199.0,6-4,225,Right,251.0,82.0
1,Drew Brees,8842.0,"['13x Pro Bowl', '1x All-Pro']",1.0,100,277,172-114-0,287,80358.0,571.0,...,243.0,2001.0,San Diego Chargers,2.0,32.0,6-0,209,Right,172.0,114.0
2,Peyton Manning,2000.0,"['14x Pro Bowl', '7x All-Pro']",2.0,100,271,186-79-0,266,71940.0,539.0,...,251.0,1998.0,Indianapolis Colts,1.0,1.0,6-5,230,Right,186.0,79.0
3,Brett Favre,1990.0,"['11x Pro Bowl', '3x All-Pro']",1.0,100,259,186-112-0,302,71838.0,508.0,...,336.0,1991.0,Atlanta Falcons,2.0,33.0,6-2,222,Right,186.0,112.0
4,Ben Roethlisberger,8842.0,['6x Pro Bowl'],2.0,100,208,165-81-1,249,64088.0,418.0,...,211.0,2004.0,Pittsburgh Steelers,1.0,11.0,6-5,240,Right,165.0,81.0


### NEW DATAFRAME:  num_cleaning_df

In [48]:
# # Fill NaN values with 0 or another appropriate value
import numpy as np
num_cleaning_df = cleaning_df.copy()

# Replace non values in 'hof' and 'team' columns
num_cleaning_df['hof'] = num_cleaning_df['hof'].replace([8842, 4423], np.nan)
num_cleaning_df['team'] = num_cleaning_df['team'].replace('Draft History', 'Undrafted')

# Calculate win percentage in new column for win_% with our new win/loss columns:
num_cleaning_df['win_%'] = round(num_cleaning_df['wins'] / (num_cleaning_df['wins'] + num_cleaning_df['loss']) * 100, 2)

# Move 'win_%' column to the position right after 'games' column
win_pct_pop = num_cleaning_df.pop('win_%')
games_index = num_cleaning_df.columns.get_loc('games')
num_cleaning_df.insert(games_index + 1, 'win_%', win_pct_pop)

display(num_cleaning_df.head())
display(num_cleaning_df.tail())
num_cleaning_df.columns

,player,hof,all_star,SBs,wAV,AV,record,games,win_%,pass_yards,...,int,drafted,team,rd,pick,ht,wt,hand,wins,loss
0,Tom Brady,2000.0,"['15x Pro Bowl', '3x All-Pro']",7.0,100,326,251-82-0,335,75.38,89214.0,...,212.0,2000.0,New England Patriots,6.0,199.0,6-4,225,Right,251.0,82.0
1,Drew Brees,NaN,"['13x Pro Bowl', '1x All-Pro']",1.0,100,277,172-114-0,287,60.14,80358.0,...,243.0,2001.0,San Diego Chargers,2.0,32.0,6-0,209,Right,172.0,114.0
2,Peyton Manning,2000.0,"['14x Pro Bowl', '7x All-Pro']",2.0,100,271,186-79-0,266,70.19,71940.0,...,251.0,1998.0,Indianapolis Colts,1.0,1.0,6-5,230,Right,186.0,79.0
3,Brett Favre,1990.0,"['11x Pro Bowl', '3x All-Pro']",1.0,100,259,186-112-0,302,62.42,71838.0,...,336.0,1991.0,Atlanta Falcons,2.0,33.0,6-2,222,Right,186.0,112.0
4,Ben Roethlisberger,NaN,['6x Pro Bowl'],2.0,100,208,165-81-1,249,67.07,64088.0,...,211.0,2004.0,Pittsburgh Steelers,1.0,11.0,6-5,240,Right,165.0,81.0


,player,hof,all_star,SBs,wAV,AV,record,games,win_%,pass_yards,...,int,drafted,team,rd,pick,ht,wt,hand,wins,loss
80,Bob Griese,1971.0,"['8x Pro Bowl', '2x All-Pro']",2.0,100,138,92-56-3,161,62.16,25092.0,...,172.0,1967.0,Miami Dolphins,1.0,4.0,6-1,190,Right,92.0,56.0
81,Tommy Kramer,NaN,['1x Pro Bowl'],0.0,100,84,54-56-0,129,49.09,24777.0,...,158.0,1977.0,Minnesota Vikings,1.0,27.0,6-2,200,Right,54.0,56.0
82,Bart Starr,1966.0,"['4x Pro Bowl', '1x All-Pro']",2.0,100,115,94-57-6,196,62.25,24718.0,...,138.0,1956.0,Green Bay Packers,17.0,200.0,6-1,197,Right,94.0,57.0
83,Charley Johnson,NaN,[],1.0,100,NaN,NaN,133,NaN,NaN,...,NaN,1994.0,Pittsburgh Steelers,1.0,17.0,6-0,200,NaN,NaN,NaN
84,Daunte Culpepper,NaN,['3x Pro Bowl'],0.0,100,98,41-59-0,105,41.00,24153.0,...,106.0,1999.0,Minnesota Vikings,1.0,11.0,6-4,260,Right,41.0,59.0


Index(['player', 'hof', 'all_star', 'SBs', 'wAV', 'AV', 'record', 'games',
       'win_%', 'pass_yards', 'TD', 'comp_%', 'YPA', 'int', 'drafted', 'team',
       'rd', 'pick', 'ht', 'wt', 'hand', 'wins', 'loss'],
      dtype='object')

In [49]:
# convert cols to int
cols_to_int = ['wins', 'loss', 'hof', 'SBs', 'games', 'TD', 'drafted', 'rd', 'pick']

# Replace blank values with NaN  
# REGEX: r: raw string, ^: beginning of string; \s: whitespace; *: zero or more occurances of preceding element; $: end of string
num_cleaning_df[cols_to_int] = num_cleaning_df[cols_to_int].replace(r'^\s*$', np.nan, regex=True)

# Convert columns to nullable integer type
# To leave values as NaN & still convert columns to int64, use pd.Int64Dtype() (allows for nullable integer columns)
num_cleaning_df[cols_to_int] = num_cleaning_df[cols_to_int].astype(pd.Int64Dtype())

display(num_cleaning_df.head())
display(num_cleaning_df.tail())
num_cleaning_df.columns


,player,hof,all_star,SBs,wAV,AV,record,games,win_%,pass_yards,...,int,drafted,team,rd,pick,ht,wt,hand,wins,loss
0,Tom Brady,2000,"['15x Pro Bowl', '3x All-Pro']",7,100,326,251-82-0,335,75.38,89214.0,...,212.0,2000,New England Patriots,6,199,6-4,225,Right,251,82
1,Drew Brees,<NA>,"['13x Pro Bowl', '1x All-Pro']",1,100,277,172-114-0,287,60.14,80358.0,...,243.0,2001,San Diego Chargers,2,32,6-0,209,Right,172,114
2,Peyton Manning,2000,"['14x Pro Bowl', '7x All-Pro']",2,100,271,186-79-0,266,70.19,71940.0,...,251.0,1998,Indianapolis Colts,1,1,6-5,230,Right,186,79
3,Brett Favre,1990,"['11x Pro Bowl', '3x All-Pro']",1,100,259,186-112-0,302,62.42,71838.0,...,336.0,1991,Atlanta Falcons,2,33,6-2,222,Right,186,112
4,Ben Roethlisberger,<NA>,['6x Pro Bowl'],2,100,208,165-81-1,249,67.07,64088.0,...,211.0,2004,Pittsburgh Steelers,1,11,6-5,240,Right,165,81


,player,hof,all_star,SBs,wAV,AV,record,games,win_%,pass_yards,...,int,drafted,team,rd,pick,ht,wt,hand,wins,loss
80,Bob Griese,1971,"['8x Pro Bowl', '2x All-Pro']",2,100,138,92-56-3,161,62.16,25092.0,...,172.0,1967,Miami Dolphins,1,4,6-1,190,Right,92,56
81,Tommy Kramer,<NA>,['1x Pro Bowl'],0,100,84,54-56-0,129,49.09,24777.0,...,158.0,1977,Minnesota Vikings,1,27,6-2,200,Right,54,56
82,Bart Starr,1966,"['4x Pro Bowl', '1x All-Pro']",2,100,115,94-57-6,196,62.25,24718.0,...,138.0,1956,Green Bay Packers,17,200,6-1,197,Right,94,57
83,Charley Johnson,<NA>,[],1,100,NaN,NaN,133,NaN,NaN,...,NaN,1994,Pittsburgh Steelers,1,17,6-0,200,NaN,<NA>,<NA>
84,Daunte Culpepper,<NA>,['3x Pro Bowl'],0,100,98,41-59-0,105,41.00,24153.0,...,106.0,1999,Minnesota Vikings,1,11,6-4,260,Right,41,59


Index(['player', 'hof', 'all_star', 'SBs', 'wAV', 'AV', 'record', 'games',
       'win_%', 'pass_yards', 'TD', 'comp_%', 'YPA', 'int', 'drafted', 'team',
       'rd', 'pick', 'ht', 'wt', 'hand', 'wins', 'loss'],
      dtype='object')

### EXPORT CSV    this time pro_ref_individ.csv

In [50]:
#Export csv
rel_path = "../Data_Artifacts/pro_ref_individ_missing.csv"
num_cleaning_df.to_csv(rel_path, index=False)

### ** ALL REMAINING CELLS: NOT USED **

In [ ]:
import pandas as pd

# reload pro ref individuals csv
rel_path = "../Data_Artifacts/pro_ref_individ.csv"
_df = pd.read_csv(rel_path)

In [ ]:
#  to_do : 
# look at the original QB Data Dataframe (with its data, and look at the values of
#  these quarterbacks) qb_name_df

#  Filter the dataframe by the list of quarterback fails and be sure no significant names were lost.
qb_fails_df = pd.DataFrame(quarterback_fails)
display(f'{len(qb_fails_df)}')
display(qb_fails_df.head())
display(qb_fails_df.info())

# display(quarterback_dict)
# display(quarterback_dict.info())

In [ ]:
import pprint
# Print the combined data for verification
print("\quarterback_dict:")
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(quarterback_dict)

### SAMPLE DATA FOR INDIVIDUAL PLAYER

<div id="info" class="players">
	<div id="meta">
<div  class="nothumb">
	<h1>
		<span>Steve Bartkowski</span>
			</h1>
<p>
  <strong>
    Steven Joseph Bartkowski
    &nbsp;(<span>Peachtree Bart</span>)
  </strong>
</p>
<p>
	<strong>Position</strong>: QB
			<strong>Throws:</strong>
		Right
</p>
<p><span>6-4</span>,&nbsp;<span>216lb</span>&nbsp;(193cm,&nbsp;97kg) </p>
<p>
  <strong>Born:</strong> 
    <span" id="necro-birth" data-birth="1952-11-12">
		<a href="/friv/birthdays.cgi?month=11&day=12">November 12</a>,&nbsp;1952
    </span>
  <span>
    in&nbsp;Des Moines,&nbsp;<a href="/friv/birthplaces.cgi?country=USA&state=IA">IA</a></span>
</p>
<p><strong>College</strong>: 
		<a href="/schools/california/">California</a>
		&nbsp;(<a href="https://www.sports-reference.com/cfb/players/steve-bartkowski-1.html">College Stats</a>)
<p><strong><a href="https://www.pro-football-reference.com/about/approximate_value.htm">Weighted Career AV (100-95-...)</a>:</strong> 69 (703rd overall since 1960)
</p>
<p><strong>High School</strong>: 
		<a href="/schools/high_schools.cgi?id=93b907c3">Buchser</a> (<a href="/schools/high_schools.cgi?hs_state=CA">CA</a>)
</p>
<p><strong>Draft</strong>: <a href="/teams/atl/1975_draft.htm">Atlanta Falcons</a> in the 1st round (1st overall) of the <a href="/years/1975/draft.htm">1975 NFL Draft</a>.</p>
</p>
</div>
</div><!-- div#meta -->

<ul id="bling">
<li class="all_star"><a href="/probowl/">2x Pro Bowl</a></li><li class=""><a href="/awards/">1975 SN RoY</a></li>
</ul>	
	<div class="uni_holder pfr">
	<a href="/players/uniform.cgi?team=atl&number=10" class="poptip default" data-tip="Atlanta Falcons 1975-1985">
	</a>
	<a href="/players/uniform.cgi?team=ram&number=10" class="poptip" data-tip="Los Angeles Rams 1986">
	</a>
	<div class="counter opener" data-class="open" data-id="info" data-control-id="meta_more_button" data-type="hide_after">+1</div>
</div>
	<span id="bling-alt-text" style="display:none;"><!--
Recognition:
* 2x Pro Bowl
* 1975 SN RoY
Uniforms:
* Number 10 for Atlanta Falcons 1975-1985
* Number 10 for Los Angeles Rams 1986
--></span>		
</div>

<div class="stats_pullout">
<div><div>
<span><strong>SUMMARY</strong></span>
<p><strong>Career</strong></p>
</div></div>
<div class="p1">
<div class="p1">
<span class="poptip" data-tip="Games played"><strong>G</strong></span>
<p>129</p></div>

<div class="p1">
<span class="poptip" data-tip="Approximate Value is our attempt to attach a single number to every player-season since 1960.<br>See our About section for more details."><strong>AV</strong></span>
<p>84</p></div>
</div><div class="p1">
<div class="p2">
<span class="poptip" data-tip="Team record in games started by this QB (regular season)"><strong>QBrec</strong></span>
<p>59-68-0</p></div>

<div class="p1">
<span class="poptip" data-tip="Percentage of Passes Completed<br>Minimum 14 attempts per scheduled game to qualify as leader.<br />Minimum 1500 pass attempts to qualify as career leader."><strong>Cmp%</strong></span>
<p>55.9</p></div>

<div class="p1">
<span class="poptip" data-tip="Yards Gained by Passing<br>For teams, sack yardage is deducted from this total"><strong>Yds</strong></span>
<p>24124</p></div>

<div class="p2">
<span class="poptip" data-tip="Yards gained per pass attempt <br>Minimum 14 attempts per scheduled game to qualify as leader.<br>Minimum 1500 pass attempts to qualify as career leader."><strong>Y/A</strong></span>
<p>7.0</p></div>

<div class="p1">
<span class="poptip" data-tip="Passing Touchdowns"><strong>TD</strong></span>
<p>156</p></div>

<div class="p2">
<span class="poptip" data-tip="Interceptions thrown"><strong>Int</strong></span>
<p>144</p></div>

<div class="p2">
<span class="poptip" data-tip="<b>Fantasy points:</b><br />
								1 point per 25 yards passing<br />
								4 points per passing touchdown<br />
								-2 points per interception thrown<br />
								1 point per 10 yards rushing/receiving<br />
								6 points per TD<br />
								2 points per two-point conversion<br />
								-2 points per fumble lost (est. prior to 1994)"><strong>FantPt</strong></span>
<p>1329.0</p></div>
</div>
</div>
</div><!-- div#info -->

In [ ]:
# For FAQ section:

# <div class="section_content" id="div_faq">
#   <h3>How old is Steve Bartkowski?</h3>
#   <p>Steve Bartkowski is 71 years old.</p>
#   <h3>When was Steve Bartkowski born?</h3>
#   <p>Steve Bartkowski was born on November 12, 1952.</p>
#   <h3>Where was Steve Bartkowski born?</h3>
#   <p>Steve Bartkowski was born in Des Moines, IA.</p>
#   <h3>How tall is Steve Bartkowski?</h3>
#   <p>Steve Bartkowski is 6-4 (193 cm) tall.</p>
#   <h3>How much did Steve Bartkowski weigh when playing?</h3>
#   <p>Steve Bartkowski weighed 216 lbs (97 kg) when playing.</p>
#   <h3>Is Steve Bartkowski in the Hall of Fame?</h3>
#   <p>Steve Bartkowski has not been elected into the Hall of Fame.</p>
#   <h3>How many games did Steve Bartkowski play?</h3>
#   <p>Steve Bartkowski played 129 games over his career.</p>
#   <h3>How many passing yards did Steve Bartkowski have?</h3>
#   <p>Steve Bartkowski had 24,124 passing yards over his career.</p>
#   <h3>How many touchdowns did Steve Bartkowski have?</h3>
#   <p>Steve Bartkowski had 156 touchdowns over his career.</p>
#   <h3>How many Super Bowls has Steve Bartkowski won?</h3>
#   <p>Steve Bartkowski has never won a Super Bowl.</p>
#   <h3>When did Steve Bartkowski retire?</h3>
#   <p>Steve Bartkowski last played in 1986.</p>
#   <h3>What are Steve Bartkowski's nicknames?</h3>
#   <p>Peachtree Bart is a nickname for Steve Bartkowski.</p>